In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    input_index = np.array(range(0, length, 24))
    label_index = input_index + 24* (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), pred_day, labels

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters

{'watershed': 'nak_auto'}

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, True, 20)

In [12]:
run_num = range(len(folder))
real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)

ddday = 31
mmmonth = 12

#length = 1

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    #df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])
    df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
                               colum_idx[idx], interpolation=interpolation_option[idx],
                              first_file_no=i, month=mmmonth, day=ddday)

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/nak/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['도개_2015.xlsx', '도개_2016.xlsx', '도개_2017.xlsx', '도개_2018.xlsx', '도개_2019.xlsx', '도개_2020.xlsx'], ['신암_2015.xlsx', '신암_2016.xlsx', '신암_2017.xlsx', '신암_2018.xlsx', '신암_2019.xlsx', '신암_2020.xlsx'], ['풍양_2015.xlsx', '풍양_2016.xlsx', '풍양_2017.xlsx', '풍양_2018.xlsx', '풍양_2019.xlsx', '풍양_2020.xlsx']]
data/nak/자동/도개_2015.xlsx
data/nak/자동/도개_2016.xlsx
data/nak/자동/도개_2017.xlsx
data/nak/자동/도개_2018.xlsx
data/nak/자동/도개_2019.xlsx
data/nak/자동/도개_2020.xlsx
time range in files :  2015-01-01 00:00  ~  2020-11-15 23:00
data/nak/자동/신암_2015.xlsx
data/nak/자동/신암_2016.xlsx
data/nak/자동/신암_2017.xlsx
data/nak/자동/신암_2018.xlsx
data/nak/자동/신암_2019.xlsx
data/nak/자동/신암_2020.xlsx
time range in files :  2015-01-01 00:00  ~  2020-11-15 23:00
data/nak/자동/풍양_2015.xlsx
data/nak/자동/풍양_2016.xlsx
data/nak/자동/풍양_2017.xlsx
data/nak/자동/풍양_2018.xlsx
data/nak/자동/풍양_2019.xlsx
data/nak/자동/풍양_2020.xlsx
time range in file

1/1 [==============================] - 0s 20ms/step - gen_loss: 10.5030 - disc_loss: 0.3120 - rmse: 0.3706 - val_loss: 0.4049
Epoch 52/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.1916 - disc_loss: 0.2881 - rmse: 0.3763 - val_loss: 0.4106
Epoch 53/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.6464 - disc_loss: 0.3036 - rmse: 0.3356 - val_loss: 0.3800
Epoch 54/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.2337 - disc_loss: 0.2996 - rmse: 0.3745 - val_loss: 0.3450
Epoch 55/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.5907 - disc_loss: 0.3074 - rmse: 0.4001 - val_loss: 0.3659
Epoch 56/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.6562 - disc_loss: 0.3111 - rmse: 0.3405 - val_loss: 0.3977
Epoch 57/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.6106 - disc_loss: 0.2990 - rmse: 0.3650 - val_loss: 0.3744
Epoch 58/2000
1/1 [===================

Epoch 110/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.4494 - disc_loss: 0.2474 - rmse: 0.3621 - val_loss: 0.3369
Epoch 111/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.7216 - disc_loss: 0.2645 - rmse: 0.2821 - val_loss: 0.2945
Epoch 112/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.2402 - disc_loss: 0.2577 - rmse: 0.2914 - val_loss: 0.3827
Epoch 113/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.8584 - disc_loss: 0.2287 - rmse: 0.3152 - val_loss: 0.3097
Epoch 114/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.5808 - disc_loss: 0.2400 - rmse: 0.3550 - val_loss: 0.2807
Epoch 115/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.9150 - disc_loss: 0.2453 - rmse: 0.3422 - val_loss: 0.2988
Epoch 116/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 5.9134 - disc_loss: 0.2520 - rmse: 0.2831 - val_loss: 0.3100
Epoch 117/2000
1/1 [

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.3422 - disc_loss: 0.2301 - rmse: 0.2910 - val_loss: 0.2869
Epoch 169/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.9406 - disc_loss: 0.2346 - rmse: 0.2730 - val_loss: 0.3010
Epoch 170/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.3781 - disc_loss: 0.2466 - rmse: 0.3688 - val_loss: 0.3031
Epoch 171/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 5.5785 - disc_loss: 0.2359 - rmse: 0.3035 - val_loss: 0.3333
Epoch 172/2000
1/1 [==============================] - 0s 22ms/step - gen_loss: 4.6720 - disc_loss: 0.2136 - rmse: 0.2744 - val_loss: 0.2980
Epoch 173/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.5216 - disc_loss: 0.2258 - rmse: 0.3160 - val_loss: 0.2526
Epoch 174/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.7319 - disc_loss: 0.2263 - rmse: 0.3058 - val_loss: 0.2740
Epoch 175/2000
1/1 [===============

1/1 [==============================] - 0s 21ms/step - gen_loss: 4.1711 - disc_loss: 0.2246 - rmse: 0.2501 - val_loss: 0.2788
Epoch 227/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 5.4494 - disc_loss: 0.2279 - rmse: 0.2916 - val_loss: 0.2814
Epoch 228/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.3558 - disc_loss: 0.2218 - rmse: 0.2875 - val_loss: 0.2845
Epoch 229/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.5567 - disc_loss: 0.2272 - rmse: 0.2823 - val_loss: 0.2682
Epoch 230/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.3788 - disc_loss: 0.2192 - rmse: 0.2262 - val_loss: 0.3117
Epoch 231/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.2141 - disc_loss: 0.2082 - rmse: 0.2913 - val_loss: 0.2772
Epoch 232/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.4422 - disc_loss: 0.2239 - rmse: 0.2582 - val_loss: 0.2647
Epoch 233/2000
1/1 [===============

...

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.4310 - disc_loss: 0.1602 - rmse: 0.1934 - val_loss: 0.2141
Epoch 1503/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.4186 - disc_loss: 0.1462 - rmse: 0.1830 - val_loss: 0.2314
Epoch 1504/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9652 - disc_loss: 0.1582 - rmse: 0.2607 - val_loss: 0.2727
Epoch 1505/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8646 - disc_loss: 0.1603 - rmse: 0.2892 - val_loss: 0.2584
Epoch 1506/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.5484 - disc_loss: 0.1615 - rmse: 0.1978 - val_loss: 0.2265
Epoch 1507/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.6332 - disc_loss: 0.1453 - rmse: 0.2314 - val_loss: 0.2119
Epoch 1508/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9197 - disc_loss: 0.1646 - rmse: 0.2283 - val_loss: 0.2278
Epoch 1509/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.6547 - disc_loss: 0.1614 - rmse: 0.2308 - val_loss: 0.2404
Epoch 1561/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.6405 - disc_loss: 0.1527 - rmse: 0.1969 - val_loss: 0.2595
Epoch 1562/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.5928 - disc_loss: 0.1579 - rmse: 0.2011 - val_loss: 0.2133
Epoch 1563/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.6815 - disc_loss: 0.1470 - rmse: 0.2360 - val_loss: 0.2951
Epoch 1564/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.7783 - disc_loss: 0.1537 - rmse: 0.2344 - val_loss: 0.2719
Epoch 1565/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8979 - disc_loss: 0.1560 - rmse: 0.2202 - val_loss: 0.2915
Epoch 1566/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.4065 - disc_loss: 0.1585 - rmse: 0.2097 - val_loss: 0.2392
Epoch 1567/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2585 - disc_loss: 0.1540 - rmse: 0.2771 - val_loss: 0.2333
Epoch 1619/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.0170 - disc_loss: 0.1559 - rmse: 0.2640 - val_loss: 0.2199
Epoch 1620/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0055 - disc_loss: 0.1516 - rmse: 0.2297 - val_loss: 0.3272
Epoch 1621/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.0361 - disc_loss: 0.1405 - rmse: 0.2417 - val_loss: 0.2305
Epoch 1622/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2450 - disc_loss: 0.1486 - rmse: 0.2471 - val_loss: 0.3074
Epoch 1623/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4772 - disc_loss: 0.1591 - rmse: 0.2604 - val_loss: 0.2447
Epoch 1624/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0903 - disc_loss: 0.1581 - rmse: 0.2569 - val_loss: 0.2520
Epoch 1625/2000
1/1 [========

1/1 [==============================] - 0s 21ms/step - gen_loss: 3.1440 - disc_loss: 0.1440 - rmse: 0.2577 - val_loss: 0.2340
Epoch 1677/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1614 - disc_loss: 0.1580 - rmse: 0.2358 - val_loss: 0.2607
Epoch 1678/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.7421 - disc_loss: 0.1617 - rmse: 0.2262 - val_loss: 0.2594
Epoch 1679/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8508 - disc_loss: 0.1491 - rmse: 0.3144 - val_loss: 0.2638
Epoch 1680/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.5365 - disc_loss: 0.1580 - rmse: 0.2021 - val_loss: 0.2121
Epoch 1681/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9858 - disc_loss: 0.1513 - rmse: 0.2788 - val_loss: 0.2732
Epoch 1682/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.7339 - disc_loss: 0.1417 - rmse: 0.2692 - val_loss: 0.2149
Epoch 1683/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8288 - disc_loss: 0.1588 - rmse: 0.2086 - val_loss: 0.2068
Epoch 1735/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1834 - disc_loss: 0.1551 - rmse: 0.3243 - val_loss: 0.2462
Epoch 1736/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3124 - disc_loss: 0.1585 - rmse: 0.2258 - val_loss: 0.2247
Epoch 1737/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.3098 - disc_loss: 0.1570 - rmse: 0.2456 - val_loss: 0.2405
Epoch 1738/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8079 - disc_loss: 0.1524 - rmse: 0.2621 - val_loss: 0.2351
Epoch 1739/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.5351 - disc_loss: 0.1427 - rmse: 0.2182 - val_loss: 0.2982
Epoch 1740/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9407 - disc_loss: 0.1519 - rmse: 0.2980 - val_loss: 0.2201
Epoch 1741/2000
1/1 [========

In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  6
out_num_features :  1


In [56]:
print(real_df_all.shape)
rnn_target_column = 'tp'
print(rnn_target_column)

#df_length = real_df_all.shape[0] - 3000
df_length = real_df_all.shape[0]

fake_df = real_df_all.iloc[:df_length,:]

(51504, 36)
tp


In [57]:
#train_df, val_df, test_df, test_df2 = dataset_slice(fake_df, 0.8, 0.1, 0.1)
#teg_check = 'train=8, val=1, test=1'

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

#val_df, test_df,train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'val=1, test=1, train=8'

#test_df, val_df, train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'test=1, val=1, train=8'

#val_df, train_df, test_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'val=1, train=8, test=1'

#test_df, train_df, val_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'test=1, train=8, val=1'

train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, test=1, val=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (36052, 36) val_df.shape :  (10301, 36) test_df.shape: (5151, 36)


In [58]:
#fake_df.shape[0]/8760

In [59]:
plt.figure()
plt.plot(real_df_all.iloc[:, out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [60]:
plt.figure()
plt.plot(val_df.iloc[:, out_features[0]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [61]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [62]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

target_col_idx :  6
out_num_features :  1


In [63]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}

In [64]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

model_path = "save/" + watershed + "models/" + pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    if not os.path.exists('save/' + watershed):
        os.makedirs('save/' + watershed)
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/nak/models/phosphorus/


In [65]:
rnn_epochs

20

## 모델 학습

In [ ]:
gru_model = model_gru(
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=50,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt")

Epoch 1/50
10/10 [==============================] - ETA: 0s - loss: 1.8050 - mean_absolute_error: 0.5442 - nse: 0.1857
Epoch 00001: val_loss improved from inf to 0.57813, saving model to save/best_model.h5
10/10 [==============================] - 8s 799ms/step - loss: 1.8050 - mean_absolute_error: 0.5442 - nse: 0.1857 - val_loss: 0.5781 - val_mean_absolute_error: 0.5521 - val_nse: 0.2128
Epoch 2/50
10/10 [==============================] - ETA: 0s - loss: 1.1817 - mean_absolute_error: 0.3577 - nse: 0.4548
Epoch 00002: val_loss improved from 0.57813 to 0.54753, saving model to save/best_model.h5
10/10 [==============================] - 8s 782ms/step - loss: 1.1817 - mean_absolute_error: 0.3577 - nse: 0.4548 - val_loss: 0.5475 - val_mean_absolute_error: 0.5402 - val_nse: 0.2513
Epoch 3/50
10/10 [==============================] - ETA: 0s - loss: 2.1914 - mean_absolute_error: 0.3341 - nse: 0.3346
Epoch 00003: val_loss improved from 0.54753 to 0.35544, saving model to save/best_model.h5
10/1

10/10 [==============================] - ETA: 0s - loss: 1.0081 - mean_absolute_error: 0.2631 - nse: 0.6212
Epoch 00022: val_loss did not improve from 0.29671
10/10 [==============================] - 8s 788ms/step - loss: 1.0081 - mean_absolute_error: 0.2631 - nse: 0.6212 - val_loss: 0.4276 - val_mean_absolute_error: 0.4309 - val_nse: 0.4203
Epoch 23/50
10/10 [==============================] - ETA: 0s - loss: 1.2985 - mean_absolute_error: 0.2771 - nse: 0.6043
Epoch 00023: val_loss did not improve from 0.29671
10/10 [==============================] - 8s 778ms/step - loss: 1.2985 - mean_absolute_error: 0.2771 - nse: 0.6043 - val_loss: 0.6270 - val_mean_absolute_error: 0.5322 - val_nse: 0.1425
Epoch 24/50
10/10 [==============================] - ETA: 0s - loss: 1.8216 - mean_absolute_error: 0.2801 - nse: 0.4187
Epoch 00024: val_loss did not improve from 0.29671
10/10 [==============================] - 8s 779ms/step - loss: 1.8216 - mean_absolute_error: 0.2801 - nse: 0.4187 - val_loss: 0.8

## core / window.py / 

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [28]:
#train_df.shape, val_df.shape, test_df.shape

In [29]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvbnfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************            
#                   SSSSSSSS               ******            ******              *****    *****                      
#        SSSS         S  SSS               ******            ******              *****    *****                     
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

NameError: name 'stop' is not defined

In [ ]:
from core.models import GRUModel, compile

In [ ]:
model112 = GRUModel(rnn_out_steps, out_num_features)
model112.load_weights(model_path+"gru.ckpt")
compile(model112)


In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    #model=model112,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    model=model112,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
##.evaluate(multi_window.train)
#model112.evaluate(multi_window.val.repeat(-1), steps=100)

In [ ]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    #print(df.shape)
    #print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    

    predictions = model(inputs).numpy()
    
    print("input : ", inputs.shape, "labels : ", labels.shape, "predictions : ", predictions.shape)
    #print(predictions.shape)
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    print("                label_day.shape : ",label_day.shape, "pred_day.shape : ", pred_day.shape)
    
    #plt.figure()
    #input_index = np.array(range(0, length, 24))
    #label_index = input_index + 24* (7 + predict_day)
    #plt.plot(input_index, inputs_day[:, 0, :], label='input')
    #plt.plot(label_index, label_day[:, predict_day, :], label='label')
    #plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    #plt.legend()
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), inputs_day, label_day, pred_day
    

In [ ]:

x = np.arange(len(val_nse))
width = 0.35
plt.figure()
plt.title(watershed + '  ['+start_year+','+end_year+']  ' + rnn_target_column)
plt.bar(x, val_pbias.values(), 0.3, label='PBIAS' )
plt.bar(x + width, val_nse.values(), 0.3, label='NSE')
plt.xticks(x,val_nse.keys(), rotation=0)
_ = plt.legend()
plt.show()
